In [1]:
import json
import pandas as pd
from ppa.pars_txo import *

In [2]:
locations = list(pd.read_csv('data/locations.csv').Location)
locations

['India',
 'Andhra_Pradesh',
 'Arunachal_Pradesh',
 'Assam',
 'Bihar',
 'Chandigarh',
 'Chhattisgarh',
 'Delhi',
 'Gujarat',
 'Haryana',
 'Himachal_Pradesh',
 'Jammu_and_Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Madhya_Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Nagaland',
 'Odisha',
 'Punjab',
 'Rajasthan',
 'Tamil_Nadu',
 'Telangana',
 'Uttar_Pradesh',
 'Uttarakhand',
 'West_Bengal']

In [3]:
for loc in locations:
    js = json.load(open(f'data/{loc}/pars_under.json', 'r'))
    targets = pd.read_csv(f'data/{loc}/targets.csv')

    attach_pars_tx(js)
    
    print(loc[:5], '\t ', sum([p['r_txs_pri'] > 0 for p in js['pars']]) / len(js['pars']))
    js['pars'] = [p for p in js['pars'] if p['r_txs_pri'] > 0]
    
    with open(f'data/{loc}/pars_txo.json', 'w') as f:
        json.dump(js, f)

India 	  0.999
Andhr 	  1.0
Aruna 	  0.7525
Assam 	  0.9815
Bihar 	  0.9965
Chand 	  0.9955
Chhat 	  0.9915
Delhi 	  0.9705
Gujar 	  1.0
Harya 	  0.9965
Himac 	  0.997
Jammu 	  0.977
Jhark 	  0.982
Karna 	  0.999
Keral 	  1.0
Madhy 	  0.9835
Mahar 	  0.995
Manip 	  0.9905
Megha 	  0.979
Nagal 	  0.946
Odish 	  0.998
Punja 	  0.9995
Rajas 	  0.998
Tamil 	  1.0
Telan 	  1.0
Uttar 	  0.9975
Uttar 	  1.0
West_ 	  0.9925
